In [1]:
import numpy as np
from setup_mnist import *
from keras.models import Input, Model
from keras.layers import Average
import tensorflow as tf
from noise_functions_dl import GradientDescentDL, gradientDescentFunc
from functools import partial
from mwu import adversary

Using TensorFlow backend.


In [2]:
data = MNIST()
X_exp = np.load("multiclass_data_2/X_exp.npy")
Y_exp = np.load("multiclass_data_2/Y_exp.npy")

In [3]:
X_exp = X_exp.reshape(-1, 28, 28, 1)
Y_exp = np.array([(np.arange(10) == l).astype(np.float32) for l in Y_exp])

In [4]:
def ensembleModels(models, model_input):
    # taken from https://medium.com/@twt446/ensemble-and-store-models-in-keras-2-x-b881a6d7693f
    # collect outputs of models in a list
    yModels=[model(model_input) for model in models] 
    # averaging outputs
    yAvg=Average()(yModels) 
    # build model from same input and avg output
    modelEns = Model(inputs=model_input, outputs=yAvg, name='ensemble')  
    return modelEns

In [5]:
alpha = 2.2
lr = .001
opt_iters=3000

with tf.Session() as sess:
    
    model_dir = "deep_networks"
    models = [conv_net(1, model_dir + "/conv1"), conv_net(0, model_dir + "/conv2"), 
              multilayer(4, 128, model_dir + "/mlp1"), multilayer(2, 256, model_dir + "/mlp2"),
              multilayer(0, 0, model_dir + "/zero_layer")]
    model_input = Input(shape=models[0].input_shape[1:])
    ensemble = ensembleModels(models, model_input)
    ensemble.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
    print ensemble.evaluate(X_exp, Y_exp)
    attack_obj = GradientDescentDL(sess, [ensemble], alpha, (28, 1, 10), -.5, .5, 
                                   targeted=False, batch_size=1, max_iterations=opt_iters,
                                   learning_rate=lr, confidence=0)
    
    noise_func = partial(gradientDescentFunc, attack=attack_obj)
    
    V = adversary(np.array([1.0]), [ensemble], X_exp, Y_exp, alpha, noise_func, False)
    
    

100/100 [==============================] - 0s 4ms/step
[0.011660071313381195, 1.0]
Number of models 1 
Iteration 2999
Time in Iteration 0.00976705551147
Norm 2.20000004768
Loss List [array([0.], dtype=float32)]
Loss [0.]
Iteration 2999
Time in Iteration 0.00902795791626
Norm 2.20000004768
Loss List [array([0.37438217], dtype=float32)]
Loss [0.37438217]
Iteration 2999
Time in Iteration 0.00877785682678
Norm 1.99584758282
Loss List [array([0.], dtype=float32)]
Loss [0.]
Iteration 2999
Time in Iteration 0.0094690322876
Norm 2.20000004768
Loss List [array([0.78464437], dtype=float32)]
Loss [0.78464437]
Iteration 2999
Time in Iteration 0.0174009799957
Norm 1.5582473278
Loss List [array([0.], dtype=float32)]
Loss [0.]
Iteration 2999
Time in Iteration 0.0105969905853
Norm 2.20000004768
Loss List [array([0.], dtype=float32)]
Loss [0.]
Iteration 2999
Time in Iteration 0.00988388061523
Norm 1.48116648197
Loss List [array([0.], dtype=float32)]
Loss [0.]
Iteration 2999
Time in Iteration 0.00972318

In [11]:
np.save("ensemble_sol_mnist_dl.npy", V)

In [16]:
model_dir = "deep_networks"
models = [conv_net(1, model_dir + "/conv1"), conv_net(0, model_dir + "/conv2"), 
          multilayer(4, 128, model_dir + "/mlp1"), multilayer(2, 256, model_dir + "/mlp2"),
          multilayer(0, 0, model_dir + "/zero_layer")]

In [17]:
[model.evaluate(X_exp + V, Y_exp) for model in models]

100/100 [==============================] - 0s 957us/step


[[0.04193442619747657, 0.99],
 [0.016358124290127306, 0.99],
 [2.619417304992676, 0.17],
 [1.4740073776245117, 0.2],
 [4.265816688537598, 0.03]]

In [19]:
min((X_exp + V).reshape(-1,))

-0.5